In [ ]:
from utils import *
from pathlib import Path

dir_suffix = ""
if (dir_suffix != "") and (not dir_suffix.endswith("/")):
    dir_suffix = dir_suffix + "/"

reference_platform_index = 0
targets_platform_index = abs(1 - reference_platform_index)

ref_time_index = 3
tgt_time_index = 3

edge_detection = True

pr_list = ["130108", "42DVK"]
# pr_list = ["164110", "163109"]
platforms = ["L8", "S2"]
# platforms = ["L8", "L4-5"]

In [ ]:
platform_dict = {"L8": "LANDSAT_8", "S2": "SENTINEL-2", "L4-5": "LANDSAT_4_5"}

output_root = Path(
    f"data/outputs/{dir_suffix}{platforms[0]}_{pr_list[0]}_{platforms[1]}_{pr_list[1]}"
)
output_root.mkdir(parents=True, exist_ok=True)
print(f"Output root: {output_root}")
sentinel_ref = platforms[reference_platform_index] == "S2"
sentinel_tgt = platforms[targets_platform_index] == "S2"
l8_ref = platforms[reference_platform_index] == "L8"
l4_5_tgt = platforms[targets_platform_index] == "L4-5"
if sentinel_ref:
    print("Using Sentinel-2 as reference platform")
if l8_ref:
    print("Using Landsat-8 as reference platform")

In [ ]:
pl1_input_pairs = Path(
    f"data/inputs/{platform_dict[platforms[0]]}_{pr_list[0]}/pairs.csv"
)
pl2_input_pairs = Path(
    f"data/inputs/{platform_dict[platforms[1]]}_{pr_list[1]}/pairs.csv"
)

pl1_df = pd.read_csv(pl1_input_pairs)
pl2_df = pd.read_csv(pl2_input_pairs)

pl_dfs = [pl1_df, pl2_df]

ref_image = pl_dfs[reference_platform_index].Reference[0]
tgt_images = [
    pl_dfs[targets_platform_index].Closest_target[0],
    pl_dfs[targets_platform_index].Farthest_target[0],
]

if sentinel_ref:
    ref_time_index = 2
    tgt_images = tgt_images[::-1]  # Reverse order for Sentinel-2
if sentinel_tgt:
    tgt_time_index = 2
if l8_ref and l4_5_tgt:
    tgt_images = tgt_images[::-1]

print(f"Reference image: {ref_image}")
print(f"Closest target image: {tgt_images[0]}")
print(f"Farthest target image: {tgt_images[1]}")


ref_time = datetime.strptime(
    os.path.basename(ref_image).split("_")[ref_time_index], "%Y%m%d"
)
tgt_times = [
    datetime.strptime(os.path.basename(tgt).split("_")[tgt_time_index], "%Y%m%d")
    for tgt in tgt_images
]
print("Time differences:", [(tgt_time - ref_time).days for tgt_time in tgt_times])
print("\n")


ref_epsg = rasterio.open(ref_image).crs.to_epsg()
print(f"Reference image EPSG: {ref_epsg}")

for i, tgt in enumerate(tgt_images):
    tgt_epsg = rasterio.open(tgt).crs.to_epsg()
    print(f"Target image {i} EPSG: {tgt_epsg}")
    if tgt_epsg != ref_epsg:
        temp_dir = output_root / "temp"
        (temp_dir / "true_color").mkdir(parents=True, exist_ok=True)
        (temp_dir / "true_color_ds").mkdir(parents=True, exist_ok=True)
        print(f"Reprojecting target image {i} to match reference EPSG {ref_epsg}...")
        reprojected_tgt = (temp_dir / "true_color") / f"{os.path.basename(tgt)}"
        if not reprojected_tgt.exists():
            reproject_tif(tgt, reprojected_tgt, rasterio.open(ref_image).crs)
            reproject_tif(
                str(tgt).replace("true_color", "true_color_ds"),
                str(reprojected_tgt).replace("true_color", "true_color_ds"),
                rasterio.open(ref_image).crs,
            )
        else:
            print(
                f"Reprojected target image {i} already exists, skipping re-projection."
            )
        tgt_images[i] = reprojected_tgt

tgt_images = [str(img) for img in tgt_images]

In [ ]:
if edge_detection:
    output_dir = output_root / "temp_edge_detection"
    process_existing_outputs(
        [ref_image] + tgt_images,
        output_dir,
        edge_detection=True,
        edge_detection_mode="canny",
        subdir="true_color",
    )

    scene_df = pd.read_csv(output_root / "temp_edge_detection/pairs.csv")
    ref_image_edge = scene_df["Reference"][0]
    tgt_images_edge = [
        scene_df["Closest_target"][0],
        scene_df["Farthest_target"][0],
    ]

In [ ]:
if edge_detection:
    fig, axes = plt.subplots(1, 3, figsize=(20, 5))
    im0 = rasterio.open(str(ref_image_edge).replace("true_color", "true_color_ds"))
    im1 = rasterio.open(str(tgt_images_edge[0]).replace("true_color", "true_color_ds"))
    im2 = rasterio.open(str(tgt_images_edge[1]).replace("true_color", "true_color_ds"))
    show(im0, ax=axes[0], cmap="gray", title=os.path.basename(ref_image_edge))
    show(im1, ax=axes[1], cmap="gray", title=os.path.basename(tgt_images_edge[0]))
    show(im2, ax=axes[2], cmap="gray", title=os.path.basename(tgt_images_edge[1]))
    plt.tight_layout()
else:
    fig, axes = plt.subplots(1, 3, figsize=(20, 5))
    im0 = rasterio.open(str(ref_image).replace("true_color", "true_color_ds"))
    im1 = rasterio.open(str(tgt_images[0]).replace("true_color", "true_color_ds"))
    im2 = rasterio.open(str(tgt_images[1]).replace("true_color", "true_color_ds"))
    show(im0, ax=axes[0], cmap="gray", title=os.path.basename(ref_image))
    show(im1, ax=axes[1], cmap="gray", title=os.path.basename(tgt_images[0]))
    show(im2, ax=axes[2], cmap="gray", title=os.path.basename(tgt_images[1]))
    plt.tight_layout()

#### Co_Register

In [ ]:
laplacian_filter = False
laplacian_for_targets_ids = None  # [1]
use_overlap = not (
    rasterio.open(ref_image).transform
    == rasterio.open(tgt_images[0]).transform
    == rasterio.open(tgt_images[1]).transform
)
print("Using overlap:", use_overlap)
print()
output_path = output_root / f"Co_Register_{platforms[reference_platform_index]}"
if laplacian_filter:
    output_path = Path(str(output_path) + "_lpc")
_, shifts, target_ids = co_register(
    ref_image,
    tgt_images,
    output_path=output_path,
    return_shifted_images=True,
    use_overlap=use_overlap,
    # phase_corr_filter=False,
    # pahse_corr_valid_num_points=1,
    # of_dist_thresh=5,
    # band_number=2,
    # rethrow_error=True,
    laplacian_kernel_size=5 if laplacian_filter else None,
    laplacian_for_targets_ids=laplacian_for_targets_ids,
)
print("\nCo-register shifts:")
for i, shift in enumerate(shifts):
    print(
        f"Target {target_ids[i]}: {tuple([np.round(el.tolist(), 3).tolist() for el in shift])} pixels"
    )

#### Karios

In [ ]:
output_dir = output_root / f"Karios_{platforms[reference_platform_index]}"
karios_executable = "/home/ubuntu/Coreg/karios/karios/karios.py"
shift_dict, target_ids = karios(
    ref_image,
    tgt_images,
    output_dir,
    karios_executable,
)
print("\nKarios shifts:")
for i, shift in enumerate(shift_dict):
    print(
        f"Target {target_ids[i]}: {tuple([np.round(el, 3).tolist() for el in shift_dict[shift]])} pixels"
    )

#### AROSICS

In [ ]:
output_dir = output_root / f"AROSICS_{platforms[reference_platform_index]}"
shifts, target_ids = arosics(
    ref_image,
    tgt_images,
    output_dir,
)
print("\nAROSICS shifts:")
for i, shift in enumerate(shifts):
    print(
        f"Target {target_ids[i]}: {tuple([np.round(el.tolist(), 3).tolist() for el in shift])} pixels"
    )

#### AROSICS EDGE

In [ ]:
if edge_detection:
    output_dir = output_root / f"AROSICS_{platforms[reference_platform_index]}_edge"
    shifts, target_ids = arosics(
        ref_image_edge,
        tgt_images_edge,
        output_dir,
        existing_ref_image=ref_image,
        existing_tgt_images=tgt_images,
    )
    print("\nAROSICS shifts:")
    for i, shift in enumerate(shifts):
        print(
            f"Target {target_ids[i]}: {tuple([np.round(el.tolist(), 3).tolist() for el in shift])} pixels"
        )